<img src= "https://cciocorrencias.com/static/images/home/cci_artesp_nobg.png"
 alt="Texto alternativo" width="400" height="100">

## **Analisador de arquivos:** *Intervenções Viárias*
---

#### **Autor da Análise:** Diego Ferreira Marques. 
#### **Tel:** (11) 3465-2413
#### **Ferramenta utilizada:** Python (Versão 3.11.3)
#### **Bibliotecas utilizadas para a análise:** Pandas (Versão 2.0.3) e Openpyxl (Versão 3.1.2)

---

#### **Importa bibliotecas para análise**

In [10]:
import pandas as pd
import openpyxl as op
from auxilio.listas import RODOVIAS, COD_SERVICO, SERVICO, DATA_INICIO, DATA_FIM, COD_NIVEL_SERVICO

#### **Leitura dos arquivos entregues e criação do dataframe**
---

#### **OBS:** *o período do arquivo precisa estar de acordo com o período da semana analisada*

In [11]:
# Digite o nome do caminho e nome do arquivo

# Exemplo: "./2023-09-11__2023-09-17/L28-ENTREVIAS/L28_PROG-SEMANAL-OBRAS-28-08-23_03-09-23.xlsx". Alterar o período conforme a obs acima
CAMINHO = "./2023-10-16__2023-10-22/L01-AUTOBAN/TESTE-AUTOBAN.xlsx"

In [12]:
# Abas do arquivo que serão analisadas
DADOS = "DADOS"
CAPA = "CAPA"

In [13]:
# Dataframe da aba CAPA
df_capa = pd.read_excel(CAMINHO, sheet_name= CAPA)

# Dataframe da aba DADOS
df_dados = pd.read_excel(CAMINHO, sheet_name= DADOS)

c:\Users\diego.marques\Desktop\PROJETOS_DIEGO\PYTHON\analise-pedagio\analisa-obras\venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


##### **Colunas necessárias para análise**

In [14]:
# Colunas que serão analisadas. A coluna APOIO PMRv não precisa estar preenchida, já que: PREENCHIDA = SIM, NÃO PREENCHIDA = NÃO

colunas_verificacao = ['LOTE', 'RODOVIA', 'CÓD. DO SERVIÇO', 'DETALHAMENTO DO SERVIÇO', 
                       'KM INICIAL', 'LAT KM INICIAL', 'LON KM INICIAL', 'KM FINAL', 'LAT KM FINAL', 'LON KM FINAL', 'SENTIDO', 
                       'DATA INÍCIO', 'HORA INÍCIO', 'DATA FIM', 'HORA FIM', 
                       'FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6', 'FIAC', 'ALT.', 
                       'EMPRESA RESPONSÁVEL', 'RESPONSÁVEL TÉCNICO PELA OPERAÇÃO VIÁRIA', 
                       'ART DO RESPONSÁVEL TÉCNICO', 'NÍVEL DE SERVIÇO C/ INTERVENÇÃO', 'MUNICÍPIO']

df_dados = df_dados[colunas_verificacao]

### **LOTE ANALISADO**

In [15]:
temp_df = df_dados.copy()

lote_avaliado = (temp_df["LOTE"].unique())
lote_avaliado = str(lote_avaliado).strip('[]')
print(f"\033[94mO LOTE QUE ESTÁ SENDO ANALISADO É O\033[0m L{lote_avaliado}")

O LOTE QUE ESTÁ SENDO ANALISADO É O L1


### **Verifica CAPA**
---

#### **Período MIN**

In [16]:
print(df_capa.iloc[18, 2])

2023-10-23 00:00:00


#### **Período MAX**

In [17]:
print(df_capa.iloc[21, 2])

2023-10-29 00:00:00


### **Análise 1: colunas com linhas em branco**
---

In [18]:
temp_df = df_dados.copy()

# Verifica se existem linhas vazias
linhas_vazias = temp_df[temp_df[colunas_verificacao].isnull().any(axis=1)] 

# Condição para verificação
if linhas_vazias.empty: 
    print("\033[92mNão foram encontradas linhas vazias. VERIFICAÇÃO OK\033[0m")
else:
    for index, row in linhas_vazias.iterrows():
        for col in colunas_verificacao:
            if pd.isnull(row[col]):
                print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Célula em branco encontrada na\033[0m (linha {index + 2}, da coluna '{col}')")
                display(linhas_vazias)

Não foram encontradas linhas vazias. VERIFICAÇÃO OK


### **Análise 2: coluna LOTE com mais de um LOTE**
---

In [19]:
temp_df = df_dados.copy()

# Cria variável com valores únicos
valores_unicos_lote = temp_df["LOTE"].unique()

# Condição para verificação
if len(valores_unicos_lote) == 1:
    print("\033[92mA coluna 'LOTE' contém apenas 1 valor. VERIFICAÇÃO OK\033[0m")  # Texto em verde
else:
    print("\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Existe mais de um valor na coluna LOTE, sendo ele(s):\033[0m", valores_unicos_lote)  # Texto em vermelho

A coluna 'LOTE' contém apenas 1 valor. VERIFICAÇÃO OK


### **Análise 3: preenchimento da coluna RODOVIA**
---

In [20]:
temp_df = df_dados.copy()

rodovia_fora = df_dados.loc[~temp_df["RODOVIA"].isin(RODOVIAS), "RODOVIA"]
valores_rodovia_fora = rodovia_fora.values.tolist()
valores_unicos_fora = set(valores_rodovia_fora)

if not valores_rodovia_fora:
    print("\033[92mA coluna 'RODOVIA' está preenchida corretamente. VERIFICAÇÃO OK\033[0m")
else:
    print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Existem campos preenchidos indevidamentes, sendo eles:\033[0m {valores_unicos_fora}")

A coluna 'RODOVIA' está preenchida corretamente. VERIFICAÇÃO OK


### **Análise 4: preenchimento da coluna CÓD. DO SERVIÇO**
---

In [21]:
temp_df = df_dados.copy()

cod_servico_fora = temp_df.loc[~temp_df["CÓD. DO SERVIÇO"].isin(COD_SERVICO), "CÓD. DO SERVIÇO"]
valores_cod_fora = cod_servico_fora.values.tolist()

if not valores_cod_fora:
    print("\033[92mA coluna 'CÓD DO SERVIÇO' está preenchida corretamente. VERIFICAÇÃO OK\033[0m")
else:
    print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Existem campos preenchidos indevidamentes, sendo eles:\033[0m {valores_cod_fora}")

A coluna 'CÓD DO SERVIÇO' está preenchida corretamente. VERIFICAÇÃO OK


### **Análise 5: DETALHAMENTO DO SERVIÇO (PENDENTE)**
---

In [22]:
# temp_df = df_dados.copy()

# servico_fora = temp_df.loc[~temp_df["DETALHAMENTO DO SERVIÇO"].isin(SERVICO), "DETALHAMENTO DO SERVIÇO"]
# valores_servico_fora = servico_fora.values.tolist()

# if not valores_servico_fora:
#     print("A coluna 'CÓD DO SERVIÇO' está preenchida corretamente. Verificação OK")
# else:
#     print(f"Existem campos preenchidos indevidamentes, sendo eles: {valores_servico_fora}")

### **Verificação: colunas DATA INICIO e DATA FIM**
---

In [23]:
df_dados["DATA INÍCIO"].unique()

<DatetimeArray>
['2023-10-23 00:00:00', '2023-10-24 00:00:00', '2023-10-25 00:00:00',
 '2023-10-26 00:00:00', '2023-10-27 00:00:00', '2023-10-28 00:00:00',
 '2023-10-29 00:00:00']
Length: 7, dtype: datetime64[ns]

In [24]:
df_dados["DATA FIM"].unique()

<DatetimeArray>
['2023-10-23 00:00:00', '2023-10-24 00:00:00', '2023-10-25 00:00:00',
 '2023-10-26 00:00:00', '2023-10-27 00:00:00', '2023-10-28 00:00:00',
 '2023-10-29 00:00:00']
Length: 7, dtype: datetime64[ns]

### **Verificação: coluna de SENTIDO**
---

In [25]:
temp_df = df_dados.copy()

SENTIDO = ['N', 'S', 'NS', 'L', 'O', 'LO', 'E', 'I', 'EI']

cod_sentido_fora = temp_df.loc[~temp_df["SENTIDO"].isin(SENTIDO), "SENTIDO"]
valores_sentido_fora = cod_sentido_fora.values.tolist()
valores_unicos_fora = set(valores_sentido_fora)

if not valores_sentido_fora:
    print("\033[92mA coluna 'SENTIDO', está preenchida corretamente. VERIFICAÇÃO OK\033[0m")
else:
    print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Existem campos preenchidos indevidamente, sendo eles:\033[0m {valores_unicos_fora}")

A coluna 'SENTIDO', está preenchida corretamente. VERIFICAÇÃO OK


### **Análise 6: preenchimento nas colunas de faixa, canteiro e acostamento**
---

In [26]:
temp_df = df_dados.copy()

# Lista de colunas para verificar
colunas_verificacao = ['FI1', 'FI2', 'FI3', 'FI4', 'FI5', 'FI6', 'FIAC', 'ALT.']

# Constante para verificação
FI = ["S", "N"]

for coluna in colunas_verificacao:

    # Coloca todos os valores das colunas de verificação para MAÍUSCULA
    temp_df[coluna] = temp_df[coluna].str.upper()
    
    valores_fora = temp_df[~temp_df[coluna].isin(FI)]

    if not valores_fora.empty:
        print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! A coluna\033[0m '{coluna}' \033[91mcontém valores fora do esperado:\033[0m")
        print(valores_fora)
    else:
        print(f"\033[92mA coluna '{coluna}' está preenchida corretamente. VERIFICAÇÃO OK\033[0m")

A coluna 'FI1' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FI2' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FI3' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FI4' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FI5' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FI6' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'FIAC' está preenchida corretamente. VERIFICAÇÃO OK
A coluna 'ALT.' está preenchida corretamente. VERIFICAÇÃO OK


### **Análise 7: preenchimento na coluna NÍVEL DE SERVIÇO**
---

In [27]:
# Cria cópia do dataframe original
temp_df = df_dados.copy()

nivel_servico_fora = temp_df.loc[~temp_df["NÍVEL DE SERVIÇO C/ INTERVENÇÃO"].isin(COD_NIVEL_SERVICO), "NÍVEL DE SERVIÇO C/ INTERVENÇÃO"]
valores_nivel_servico_fora = nivel_servico_fora.values.tolist()

if not valores_nivel_servico_fora:
    print("\033[92mA coluna 'NÍVEL DE SERVIÇO' está preenchida corretamente. VERIFICAÇÃO OK\033[0m")
else:
    print(f"\033[91mANÁLISE CONTÉM DIVERGÊNCIAS!! Existem campos preenchidos indevidamentes, sendo eles:\033[0m {valores_nivel_servico_fora}")

A coluna 'NÍVEL DE SERVIÇO' está preenchida corretamente. VERIFICAÇÃO OK
